In [158]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.core.display import display, HTML


In [121]:
# import files
death = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
confirmed = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
country = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')

In [122]:
#death.head()

In [123]:
#confirmed.head()

In [124]:
#recovered.head()

In [125]:
#countries.head()

In [126]:
# Cleaning the data
countries.columns = map(str.lower, countries.columns)
recovered.columns = map(str.lower, recovered.columns)
death.columns = map(str.lower, death.columns)
confirmed.columns = map(str.lower, confirmed.columns)

In [127]:
confirmed = confirmed.rename(columns = {'province/state': 'state', 'country/region': 'country'})
recovered = recovered.rename(columns = {'province/state': 'state', 'country/region': 'country'})
death = death.rename(columns = {'province/state': 'state', 'country/region': 'country'})
countries = countries.rename(columns={'country_region': 'country'})
                                        

In [128]:
sort_countries = countries.sort_values('confirmed', ascending=False).head(10)


In [129]:
sort_countries

,country,last_update,lat,long_,confirmed,deaths,recovered,active,incident_rate,people_tested,people_hospitalized,mortality_rate,uid,iso3
17,US,2020-05-19 18:32:23,40.000000,-100.000000,1519986.0,91179.0,283178.0,1143265.0,461.347967,NaN,NaN,5.998674,840,USA
13,Russia,2020-05-19 18:32:23,61.524000,105.318800,299941.0,2837.0,76130.0,220974.0,205.531305,NaN,NaN,0.945853,643,RUS
40,Brazil,2020-05-19 18:32:23,-14.235000,-51.925300,262545.0,17509.0,100459.0,144577.0,123.516057,NaN,NaN,6.668952,76,BRA
16,United Kingdom,2020-05-19 18:32:23,55.000000,-3.000000,250121.0,35421.0,1094.0,213606.0,368.442662,NaN,NaN,14.161546,826,GBR
18,Spain,2020-05-19 18:32:23,40.463667,-3.749220,232037.0,27778.0,0.0,204259.0,496.285054,NaN,NaN,11.971367,724,ESP
10,Italy,2020-05-19 18:32:23,41.871900,12.567400,226699.0,32169.0,129401.0,65129.0,374.945660,NaN,NaN,14.190182,380,ITA
6,France,2020-05-19 18:32:23,46.227600,2.213700,180933.0,28025.0,62678.0,90230.0,277.192071,NaN,NaN,15.489159,250,FRA
7,Germany,2020-05-19 18:32:23,51.165691,10.451526,177574.0,8081.0,155681.0,13812.0,211.942753,NaN,NaN,4.550779,276,DEU
175,Turkey,2020-05-19 18:32:23,38.963700,35.243300,151615.0,4199.0,112895.0,34521.0,179.768410,NaN,NaN,2.769515,792,TUR
91,Iran,2020-05-19 18:32:23,32.427908,53.688046,124603.0,7119.0,97173.0,20311.0,148.349350,NaN,NaN,5.713346,364,IRN


In [130]:
confirmed_total = int(countries['confirmed'].sum())
deaths_total = int(countries['deaths'].sum())
recovered_total = int(countries['recovered'].sum())
active_total = int(countries['recovered'].sum())

In [131]:
display(HTML("<div style = 'background-color: #504e4e; padding: 30px '>" +
             "<span style='color: #fff; font-size:30px;'> Confirmed: "  + str(confirmed_total) +"</span>" +
             "<span style='color: red; font-size:30px;margin-left:20px;'> Deaths: " + str(deaths_total) + "</span>"+
             "<span style='color: lightgreen; font-size:30px; margin-left:20px;'> Recovered: " + str(recovered_total) + "</span>"+
             "</div>")
       )

In [132]:
def highlight_col(x):
    r = 'background-color: red'
    p = 'background-color: purple'
    gr = 'background-color: green'
    temp = pd.DataFrame('', index=x.index, columns = x.columns)
    temp.iloc[:, 4] = p
    temp.iloc[:, 5] = r
    temp.iloc[:, 6] = gr
    return temp
    
sort_countries.style.apply(highlight_col, axis=None)

,country,last_update,lat,long_,confirmed,deaths,recovered,active,incident_rate,people_tested,people_hospitalized,mortality_rate,uid,iso3
17,US,2020-05-19 18:32:23,40,-100,1.51999e+06,91179,283178,1.14326e+06,461.348,nan,nan,5.99867,840,USA
13,Russia,2020-05-19 18:32:23,61.524,105.319,299941,2837,76130,220974,205.531,nan,nan,0.945853,643,RUS
40,Brazil,2020-05-19 18:32:23,-14.235,-51.9253,262545,17509,100459,144577,123.516,nan,nan,6.66895,76,BRA
16,United Kingdom,2020-05-19 18:32:23,55,-3,250121,35421,1094,213606,368.443,nan,nan,14.1615,826,GBR
18,Spain,2020-05-19 18:32:23,40.4637,-3.74922,232037,27778,0,204259,496.285,nan,nan,11.9714,724,ESP
10,Italy,2020-05-19 18:32:23,41.8719,12.5674,226699,32169,129401,65129,374.946,nan,nan,14.1902,380,ITA
6,France,2020-05-19 18:32:23,46.2276,2.2137,180933,28025,62678,90230,277.192,nan,nan,15.4892,250,FRA
7,Germany,2020-05-19 18:32:23,51.1657,10.4515,177574,8081,155681,13812,211.943,nan,nan,4.55078,276,DEU
175,Turkey,2020-05-19 18:32:23,38.9637,35.2433,151615,4199,112895,34521,179.768,nan,nan,2.76951,792,TUR
91,Iran,2020-05-19 18:32:23,32.4279,53.688,124603,7119,97173,20311,148.349,nan,nan,5.71335,364,IRN


In [133]:
import plotly.express as px

In [115]:
fig = px.scatter(sort_countries.head(10), x='country', y='confirmed', size='confirmed', color='country', hover_name='country', size_max=60)


fig.show()

In [138]:
import plotly.graph_objects as go

def plot_case(country):
    labels = ['confirmed', 'deaths']
    colors = ['blue', 'red']
    mode_size = [6, 8]
    line_size = [4, 5]

    df_list = [confirmed, death]
    fig = go.Figure()
    
    for i, df in enumerate(df_list):
        if country == 'World' or country == 'world':
            x_data = np.array(list(df.iloc[:, 5:].columns))
            y_data = np.sum(np.asarray(df.iloc[:, 5:]), axis = 0)
        
        else:
            x_data = np.array(list(df.iloc[:, 5:].columns))
            y_data = np.sum(np.asarray(df[df['country'] == country].iloc[:, 5:]), axis = 0)
            
        fig.add_trace(go.Scatter(x=x_data, y = y_data, mode='lines+markers',
                                name=labels[i],
                                line=dict(color=colors[i], width=line_size[i]),
                                connectgaps=True,
                                text = "Total " + str(labels[i]) + ": " + str(y_data[-1])
                                ))
        
        fig.update_layout(title_text = "COVID-19 Cases of " + country,
                         xaxis_title='Date',
                         yaxis_title='No. of Confirmed Cases',
                         font=dict(family="Arial, monospace", size = 20))
        
        fig.show()
plot_case('US')
        

# World Map

### Interactive Map

In [139]:
import folium

In [154]:
world_map = folium.Map(location=[11,0], tiles='cartodbpositron', zoom_start=2, max_zoom = 6, min_zoom = 2)

for i in range(len(confirmed)):
    folium.Circle(
        location=[confirmed.iloc[i]['lat'], confirmed.iloc[i]['long']],
        fill = True,
        radius = (int((np.log(confirmed.iloc[i, -1]+1.00001))) + 0.2)*50000,
        fill_color = 'blue',
        color = 'red',
        tooltip = "<div style='margin: 0; background-color: aquamarine; color: black;'>"+
                    "<h4 style='text-align:center;font-weight: bold'>"+confirmed.iloc[i]['country'] + "</h4>"
                    "<hr style='margin:10px;color: black;'>"+
                    "<ul style='color: black;;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
                        "<li>Confirmed: "+str(confirmed.iloc[i,-1])+"</li>"+
                        "<li>Deaths:   "+str(death.iloc[i,-1])+"</li>"+
                        "<li>Death Rate: "+ str(np.round(death.iloc[i,-1]/(confirmed.iloc[i,-1]+1.00001)*100,2))+ "</li>"+
                    "</ul></div>",
        ).add_to(world_map)
    
world_map
                

# Worst Hit Countries

In [176]:
px.bar(
    sort_countries.head(10),
    x = "country",
    y = "confirmed",
    title = "Top 10 Worst Affected Countries - Confirmed Cases",
    color_discrete_sequence=['cornflowerblue'],
    height = 500, 
    width = 800
)

In [177]:
px.bar(
    sort_countries.head(10),
    x = "country",
    y = "deaths",
    title = "Top 10 Worst Affected Countries - Deaths",
    color_discrete_sequence=['mediumseagreen'],
    height = 500, 
    width = 800
)


In [179]:
px.bar(
    sort_countries.head(10),
    x = "country",
    y = "recovered",
    title = "Top 10 Worst Affected Countries - Recovered Cases",
    color_discrete_sequence=['salmon'],
    height = 500, 
    width = 800
)